In [1]:
import glob
import re
import pandas as pd

In [2]:
data_dir = '../data/'

In [3]:
files = glob.glob(data_dir + 'etl_s2_v2_*.parquet')
files

['../data/etl_s2_v2_1680419520_1680590400.parquet',
 '../data/etl_s2_v2_1680112320_1680265920.parquet',
 '../data/etl_s2_v2_1679805120_1679958720.parquet',
 '../data/etl_s2_v2_1679958720_1680112320.parquet',
 '../data/etl_s2_v2_1680590400_1680744000.parquet',
 '../data/etl_s2_v2_1679178240_1679331840.parquet',
 '../data/etl_s2_v2_1680265920_1680419520.parquet',
 '../data/etl_s2_v2_1679485440_1679651520.parquet',
 '../data/etl_s2_v2_1679651520_1679805120.parquet',
 '../data/etl_s2_v2_1679331840_1679485440.parquet']

In [4]:
tickers = ['BTC-USD','ETH-USD','MKR-USD','BCH-USD','COMP-USD','AAVE-USD','UNI-USD','CRV-USD','BAL-USD','LTC-USD','ADA-USD','XLM-USD','ALGO-USD','ZEC-USD','DOGE-USD','SHIB-USD','SUSHI-USD','EOS-USD','ETC-USD','WBTC-USD']

In [5]:
data = pd.read_parquet(files[0])
columns = data.columns

In [6]:
def get_ticker_columns(ticker, columns, const_columns):
    cols = [col for col in columns if col.startswith(ticker) or col in const_columns]
    return cols

In [7]:
def get_column_field(column, const_columns):
    if column in const_columns:
        return column
    m = re.match(r'[a-zA-Z-]+:(.*)$', column)
    return m.group(1)

In [8]:
def get_all_data(files, columns, tickers, const_columns):
    all_tickers = []
    for ticker in tickers:
        tcolumns = get_ticker_columns(ticker, columns, const_columns)
        colmap = {col : get_column_field(col, const_columns) for col in tcolumns}
        dfs = []
        for file in files:
            df = pd.read_parquet(file, columns=tcolumns)
            df = df.rename(columns=colmap)
            dfs.append(df)
        df = pd.concat(dfs)
        df.sort_index(inplace=True)
        df['ticker'] = ticker
        all_tickers.append(df)
    out = pd.concat(all_tickers)
    out.sort_index(inplace=True)
    return out

In [9]:
data = get_all_data(files[:6], columns, tickers, ['book_mean_return_27', 'sequence_interval_s'])

In [10]:
data.shape

(18361680, 711)

In [11]:
row_count = data.shape[0]

In [12]:
nas = data.isna().sum()

In [13]:
nas.sum()

1305482105

In [16]:
book_col_count = 0
trade_col_count = 0
for (count, name) in zip(nas, nas.index):
    nan_percent = float(count) / row_count
    if nan_percent > 0.45:
        print(f"{name}: {count}")
    if nan_percent > 0.45 and 'trade' not in name:
        book_col_count += 1
    if nan_percent > 0.45 and 'trade' in name:
        trade_col_count += 1
print(f"total book column count: {book_col_count}")
print(f"total trade column count: {trade_col_count}")

bid_avg_tick_324skew: 8273175
bid_avg_tick_648kurt: 8273175
bid_avg_tick_3skew: 8273175
bid_avg_tick_9kurt: 8273175
trade_price_slope_9: 12488249
bid_avg_tick_960kurt: 8273175
trade_price_slope_27: 8483323
bid_avg_tick_81skew: 8273175
bid_avg_tick_27kurt: 8273175
trade_price_rmse_9: 12779680
trade_size_9std: 9493132
bid_avg_tick_960skew: 8273175
bid_avg_tick_81kurt: 8273175
trade_avg_price: 15597012
trade_return: 15597023
bid_avg_tick_3kurt: 8273175
bid_avg_tick_9skew: 8273175
bid_avg_tick_27skew: 8273175
bid_avg_tick_162skew: 8273175
bid_avg_tick_162kurt: 8273175
bid_avg_tick_324kurt: 8273175
bid_avg_tick_648skew: 8273175
trade_avg_price_9avg: 9493132
trade_volume_9std: 9493132
trade_avg_price_9std: 9493132
trade_return_9sum: 11850595
trade_return_9std: 11850595
trade_price_rmse_27: 8748741
total book column count: 16
total trade column count: 12


In [21]:
data[['sequence_interval_s','ticker']].groupby('sequence_interval_s').count()

,ticker
sequence_interval_s,
1,18357640
2,1940
3,520
4,280
5,200
6,100
7,80
8,20
9,40
